**House Prices No.2**

This file is split into the following steps:


1.   Imports (including Data)
2.   Cleaning the Data
3.   Building a model
4.   Training the model
5.   Make predictions


**1. Imports**



In [0]:
# General imports:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import Data:
train = pd.read_csv('./train.csv')
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [0]:
# Remove 'Id' and split train into train and train_labels ('SalePrice')
train.pop('Id')
train_labels = train.pop('SalePrice')

# Let's keep track of all the variables that we remove from the data set, since
# we'll have to do exactly the same for the test set:
total_dropped = ['Id']

train.shape

(1460, 79)

In [0]:
train_labels[:5]

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

**2. Cleaning the Data**

Now we can clean the data. One good start may be to **eliminate columns which are mostly empty**. Another consideration is, **whether the text-based data is really relevant**. If so, we should transform it such that we no longer have texts (categories) but additional binary columns (e.g. Country: { USA, Switzerland, Iran } -> USA: {0, 1}, Switzerland: {0, 1}, Iran: {0, 1}). If not, we might as well just drop these columns. Next: If two or more **columns highly correlate, we can drop one** for the other, because they essentially stand for the same construct. Lastly, we might want to **normalize our data** - that is: pack all values between 0 and 1.

In [0]:
# Eliminate columns which are mostly empty:
#    Identify them first:
empty = pd.concat([train.isnull().sum(), train.isnull().sum() / train.shape[0]],
                  axis=1, keys=['#NaN', '%-age'])
empty[empty.sum(axis=1) > 0]

,#NaN,%-age
LotFrontage,259,0.177397
Alley,1369,0.937671
MasVnrType,8,0.005479
MasVnrArea,8,0.005479
BsmtQual,37,0.025342
BsmtCond,37,0.025342
BsmtExposure,38,0.026027
BsmtFinType1,37,0.025342
BsmtFinType2,38,0.026027
Electrical,1,0.000685


In [0]:
#    eliminate the columns where more than 6% are missing information
to_drop = empty[empty['%-age'] > 0.06]
to_drop_list = to_drop.index.tolist()

train.drop(to_drop_list, axis=1, inplace=True)

# Let's add the dropped items to our complete list:
for item in to_drop_list:
  total_dropped.append(item)

train.shape

(1460, 73)

In [0]:
# identify text-based elements
dtypes = train.dtypes
dtypes_nonnumeric = dtypes[dtypes == 'O']
dtypes_nonnumeric.index.tolist()

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

The ones that seem to make sense for this task (as judged based on data_description.txt) are:


1.   MSZoning
2.   Street
3.   Land Contour
4.   Slope of Property
5.   Type of dwelling
6.   Type of roof
7.   ExterCond
8.   BsmtCond
9.   Heating
10. KitchenQual
11. Functional
12. SaleType

What this means is, that we have to add binary features for each category - which I might come back and do at some point...

In [0]:
#    for now: just erase all of them:
train.drop(dtypes_nonnumeric.index.tolist(), axis=1, inplace=True)
train.shape

# Let's keep track of the dropped items:
for item in dtypes_nonnumeric.index.tolist():
  total_dropped.append(item)

Next: What correlation can we identify? Are some variables synonymous for each other?

In [0]:
# Control for correlation and remove unnecessary features
corrmat = train.corr()
corrmat[corrmat.abs()>0.75]


,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
MSSubClass,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LotArea,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OverallQual,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OverallCond,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YearBuilt,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.825667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YearRemodAdd,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MasVnrArea,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BsmtFinSF1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BsmtFinSF2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BsmtUnfSF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can see in our table (sorry, not very computer sciency...) that the following variables correlate:



> GarageYrBlt  -  **YearBuilt**

> **GarageArea**  -  GarageCars

> GrLivArea      -  **TotRmsAbvGrd**

> 1stFlrSF         -  **TotalBsmtSF**

Of these, the bold ones seem to be more generally relevant, so let's erase the other ones: [GarageYrBlt, GarageCars, GrLivArea, 1stFlrSF]

In [0]:
to_drop = ['GarageYrBlt', 'GarageCars', 'GrLivArea', '1stFlrSF']
train.drop(to_drop, axis=1, inplace=True)

for item in to_drop:
  total_dropped.append(item)

Also, there is a column that still features NaN values, which (as I know now, after having run the model unsuccessfully) messes with the entire network. Hence, it has to be eliminated (or replaced, but really, I don't mind erasing it).

In [0]:
train.isnull().sum()

MSSubClass       0
LotArea          0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
MasVnrArea       8
BsmtFinSF1       0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
2ndFlrSF         0
LowQualFinSF     0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
TotRmsAbvGrd     0
Fireplaces       0
GarageArea       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
MiscVal          0
MoSold           0
YrSold           0
dtype: int64

In [0]:
to_drop = 'MasVnrArea'
train.drop(to_drop, axis=1, inplace=True)

total_dropped.append(to_drop)

Let's also drop all the weird classes that probably don't add much value:

In [0]:
# ['MSSubClass', 'BsmtFullBath', 'BsmtHalfBath', 'MiscVal', 'MoSold', 'YrSold']
to_drop = ['MSSubClass', 'BsmtFullBath', 'BsmtHalfBath', 'MiscVal', 'MoSold',
          'YrSold', 'LowQualFinSF', 'BsmtFinSF2', 'ScreenPorch', 'PoolArea',
          'EnclosedPorch', '3SsnPorch', 'BsmtFinSF1', 'BsmtUnfSF', 'BedroomAbvGr',
          'WoodDeckSF']
train.drop(to_drop, axis=1, inplace=True)

for item in to_drop:
  total_dropped.append(item)

Finally, let's **normalize** our data. That is: We should bring all values to a within-column range of 0-1. To do so, I use a min-max normalization, that is: 

y = (x - min) / (max - min)

In [0]:
train_normalized = (train - train.min())/(train.max() - train.min())

train_normalized.head()

,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,TotalBsmtSF,2ndFlrSF,FullBath,HalfBath,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea,OpenPorchSF
0,0.033420,0.666667,0.500,0.949275,0.883333,0.140098,0.413559,0.666667,0.5,0.333333,0.500000,0.000000,0.386460,0.111517
1,0.038795,0.555556,0.875,0.753623,0.433333,0.206547,0.000000,0.666667,0.0,0.333333,0.333333,0.333333,0.324401,0.000000
2,0.046507,0.666667,0.500,0.934783,0.866667,0.150573,0.419370,0.666667,0.5,0.333333,0.333333,0.333333,0.428773,0.076782
3,0.038561,0.666667,0.500,0.311594,0.333333,0.123732,0.366102,0.333333,0.0,0.333333,0.416667,0.333333,0.452750,0.063985
4,0.060576,0.777778,0.500,0.927536,0.833333,0.187398,0.509927,0.666667,0.5,0.333333,0.583333,0.333333,0.589563,0.153565


**3.   Build the model**

At this point, we stop manipulating the input data. We have to go through the exact same process with our test data later on. For now, let's build a model. I think going for 64 neurons per hidden layer and using two of the latter should do the trick. However, I have no idea if that is adequate. The output layer has one neuron - which is essentially the SalePrice. Notice how the last layer doesn't have to carry an activation function. We don't want it to 'decide' on categories, but to freely predict some (float) value for a house.

- Optimizer: tf.keras.optimizers.RMSprop(0.001)     (no idea what other options there are)
- Loss-function: 'mean_squared_error'
- Metrics: ['mean_absolute_error', 'mean_squared_error']

In [0]:
def build_model():
  model = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu, 
                       input_shape=[len(train_normalized.keys())]),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(1)
  ])
  
  model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
             loss='mean_squared_error',
             metrics=['mean_squared_error'])

  return model

model = build_model()

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                960       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 5,185
Trainable params: 5,185
Non-trainable params: 0
_________________________________________________________________


**4.   Train the model**

Let's now use our training set and jog the model. Before going all the way, we should check if the output of the model is what we expect it to be:

In [0]:
# use the untrained model to predict some of the raining data's labels
example_batch = train[:5]
example_result = model.predict(example_batch)
example_result

array([[  66.789185],
       [ -84.170166],
       [-183.6731  ],
       [ -65.35663 ],
       [-383.6328  ]], dtype=float32)

The output seems fine. Strangely enough, the output values are lists with one item (why?), but we can deal with that. The actual training is next. Let's go for 5'000 epochs. To make sure the program is not stuck, let's get some update on how it is doing by printing dots - one every 100 epochs.

In [0]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 4000

history = model.fit(
  train_normalized, (train_labels - train_labels.min())/(train_labels.max() - train_labels.min()),
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])


Instructions for updating:
Use tf.cast instead.

....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
..........................................

In [0]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,mean_squared_error,val_loss,val_mean_squared_error,epoch
3995,0.000064,0.000064,0.004213,0.004213,3995
3996,0.000063,0.000063,0.003953,0.003953,3996
3997,0.000070,0.000070,0.004324,0.004324,3997
3998,0.000060,0.000060,0.003820,0.003820,3998
3999,0.000074,0.000074,0.003849,0.003849,3999


Let's consider the model to be trained now. Notice: I have also normalized the labels. In other words, we have to undo this transformation for our predictions:

In [0]:
example_batch = train_normalized[:5]
example_result = model.predict(example_batch)
example_result * (train_labels.max() - train_labels.min()) + train_labels.min()

array([[198511.3351047 ],
       [184574.91623163],
       [215760.16274691],
       [137387.05142587],
       [254799.87925887]])

**5. Make Predictions**

For our predicitons, we have to do the following now:


1.   Get the test data
2.   Remove all dropped lines
3.   Normalize the data
4.   Run the model
5.   Reverse the normalization of the results

In [0]:
# 1. Get the test data
test = pd.read_csv('./test.csv')
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [0]:
total_dropped

['Id',
 'LotFrontage',
 'Alley',
 'FireplaceQu',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition',
 'GarageYrBlt',
 'GarageCars',
 'GrLivArea',
 '1stFlrSF',
 'MasVnrArea',
 'MSSubClass',
 'BsmtFullBath',
 'BsmtHalfBath',
 'MiscVal',
 'MoSold',
 'YrSold',
 'LowQualFinSF',
 'BsmtFinSF2',
 'ScreenPorch',
 'PoolArea',
 'EnclosedPorch',
 '3SsnPorch',
 'BsmtFinSF1',
 'BsmtUnfSF',
 'BedroomAbvGr',
 'WoodDeckSF']

In [0]:
# Remove the dropped lines:
test.drop(total_dropped, axis=1, inplace=True)

test.shape

(1459, 14)

In [0]:
# check if the number of columns match:
train.shape

(1460, 14)

In [0]:
# normalize the data:
test = (test - test.min())/(test.max() - test.min())

test.head()

,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,TotalBsmtSF,2ndFlrSF,FullBath,HalfBath,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea,OpenPorchSF
0,0.184147,0.444444,0.625,0.625954,0.183333,0.173111,0.000000,0.25,0.0,0.5,0.166667,0.00,0.490591,0.000000
1,0.232124,0.555556,0.625,0.603053,0.133333,0.260844,0.000000,0.25,0.5,0.5,0.250000,0.00,0.209677,0.048518
2,0.224197,0.444444,0.500,0.900763,0.800000,0.182139,0.376477,0.50,0.5,0.5,0.250000,0.25,0.323925,0.045822
3,0.154326,0.555556,0.625,0.908397,0.800000,0.181747,0.364125,0.50,0.5,0.5,0.333333,0.25,0.315860,0.048518
4,0.064121,0.777778,0.500,0.862595,0.700000,0.251227,0.000000,0.50,0.0,0.5,0.166667,0.00,0.340054,0.110512


In [0]:
# run the model:
results_normalized = model.predict(test)

results_normalized

array([[0.1930423 ],
       [0.20315322],
       [0.22824797],
       ...,
       [0.13335608],
       [0.10923095],
       [0.3140754 ]], dtype=float32)

In [0]:
# reverse the normalization:
results = results_normalized * (train_labels.max() - train_labels.min()) + train_labels.min()

results

array([[173909.75532681],
       [181190.63573778],
       [199261.36344373],
       ...,
       [130929.71281856],
       [113557.20743686],
       [261065.7030046 ]])

Lastly we have to export the data such that it fits the right formatting, which looks somewhat like this:
'Id'       'SalePrice'
1460    124256.4324
...           ...

In [0]:
# 1. Get the 'Id' row
output = pd.read_csv('./test.csv')
output = output['Id']


# 2. adapt the shape of the results
results_list = []
for result in results.tolist():
  results_list.append(float(result[0]))

output = pd.DataFrame({'Id': output, 'SalePrice': results_list})

output['Id'].dtype

dtype('int64')

In [0]:
output.describe()

,Id,SalePrice
count,1459.000000,1457.000000
mean,2190.000000,201747.476643
std,421.321334,94366.954841
min,1461.000000,-106451.724127
25%,1825.500000,141061.962804
50%,2190.000000,179372.714607
75%,2554.500000,252946.430019
max,2919.000000,563995.901942


In [0]:
# 3. Clean out the NaN datapoints, and also negative values:
output = output.replace(np.nan, output['SalePrice'].mean())
output['SalePrice'][output['SalePrice'] < 0] = output['SalePrice'].mean()
output.describe()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Id,SalePrice
count,1459.000000,1459.000000
mean,2190.000000,202829.737176
std,421.321334,92958.251755
min,1461.000000,3240.929107
25%,1825.500000,141885.692389
50%,2190.000000,180030.258265
75%,2554.500000,252789.960402
max,2919.000000,563995.901942


In [0]:
# 4. Write output file
fout = open('./submission.csv', "w")

output.to_csv(fout, sep=',', header=True, index=False)